<a href="https://colab.research.google.com/github/jchburmester/ann_tf/blob/main/homework_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 10**

In [144]:
"""Cell for imports & variable settings"""

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nltk
import tqdm
import re
import string
from tensorflow.keras import layers

# Load the TensorBoard notebook extension
%load_ext tensorboard

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [130]:
"""Reading data and print some examples"""

with open('bible.txt') as file:
    lines = file.read().splitlines()
for line in lines[:20]:
  print(line)

The First Book of Moses:  Called Genesis


1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light: and there was light.

1:4 And God saw the light, that it was good: and God divided the light
from the darkness.

1:5 And God called the light Day, and the darkness he called Night.
And the evening and the morning were the first day.

1:6 And God said, Let there be a firmament in the midst of the waters,
and let it divide the waters from the waters.



## Pre-processing

In [139]:
"""Cell for generating training data"""
  
def gen_train(sequences, window_size, num_ns, vocab_size, seed):  
  
  # elements of each training example are appended to these lists
  targets, contexts, labels = [], [], []

  # build sampling table for vocab_size tokens
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=vocab_size)
  
  # iterate over all sequences (sentences) in dataset
  for sequence in tqdm.tqdm(sequences):

    # generate positive skip-gram pairs for a sequence (sentence)
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
        sequence,
        vocabulary_size=vocab_size,
        window_size=window_size,
        negative_samples=0)
    
    # iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # append each element from the training example to global lists
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [133]:
"""Construct a TextLineDataset object"""

text_ds = tf.data.TextLineDataset('bible.txt').filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [136]:
"""Cell for pre-processing"""

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase, 
                                  '[%s]' % re.escape(string.punctuation), '')

# define the vocabulary size and number of words in a sequence
vocab_size = 4096
sequence_length = 10

# normalize, split, and map strings to integers
# set output_sequence_length length to pad all samples to same length
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

#  create vocabulary
vectorize_layer.adapt(text_ds.batch(1024))

# save the created vocabulary for reference
inverse_vocab = vectorize_layer.get_vocabulary()

# vectorize the data in text_ds
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

# obtain sequences from dataset
sequences = list(text_vector_ds.as_numpy_iterator())

In [126]:
"""Inspect some examples"""

for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[  2 219 404   4 133 162   1   0   0   0] => ['the', 'first', 'book', 'of', 'moses', 'called', '[UNK]', '', '', '']
[1003    7    2  684   28 1424    2  173    3    2] => ['11', 'in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the']
[1002    3    2  111   27  223 2399    3 2366    3] => ['12', 'and', 'the', 'earth', 'was', 'without', 'form', 'and', 'void', 'and']
[   2  230    4    2 1010    3    2  191    4   28] => ['the', 'face', 'of', 'the', 'deep', 'and', 'the', 'spirit', 'of', 'god']
[302   0   0   0   0   0   0   0   0   0] => ['waters', '', '', '', '', '', '', '', '', '']


In [140]:
"""Generate training examples from sequences"""

targets, contexts, labels = gen_train(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 74644/74644 [06:54<00:00, 180.00it/s]




targets.shape: (2160808,)
contexts.shape: (2160808, 5)
labels.shape: (2160808, 5)


In [141]:
"""Configure dataset for performance"""

BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


## Model and Training

In [142]:
"""Create a Word2Vec class using keras"""

class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair

    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
      
    # target: (batch,)
    word_emb = self.target_embedding(target)

    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)

    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    
    # dots: (batch, context)
    return dots

In [146]:
"""Define loss and compile model"""

num_ns = 4
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [147]:
"""Training the model"""

word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
2110/2110 [==============================] - 40s 19ms/step - loss: 1.3294 - accuracy: 0.4597
Epoch 2/20
2110/2110 [==============================] - 33s 16ms/step - loss: 1.2014 - accuracy: 0.5161
Epoch 3/20
2110/2110 [==============================] - 36s 17ms/step - loss: 1.1541 - accuracy: 0.5364
Epoch 4/20
2110/2110 [==============================] - 34s 16ms/step - loss: 1.1246 - accuracy: 0.5485
Epoch 5/20
2110/2110 [==============================] - 34s 16ms/step - loss: 1.1029 - accuracy: 0.5574
Epoch 6/20
2110/2110 [==============================] - 34s 16ms/step - loss: 1.0854 - accuracy: 0.5641
Epoch 7/20
2110/2110 [==============================] - 34s 16ms/step - loss: 1.0706 - accuracy: 0.5699
Epoch 8/20
2110/2110 [==============================] - 34s 16ms/step - loss: 1.0577 - accuracy: 0.5749
Epoch 9/20
2110/2110 [==============================] - 33s 16ms/step - loss: 1.0463 - accuracy: 0.5790
Epoch 10/20
2110/2110 [==============================] - 34s 16m